In [2]:
!pip install imutils

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25860 sha256=b3ff96fb67cb75553387aaa8aaabe51ed7772e7f3aca78c39fbc61a5e1ebfef1
  Stored in directory: c:\users\sns\appdata\local\pip\cache\wheels\86\d7\0a\4923351ed1cec5d5e24c1eaf8905567b02a0343b24aa873df2
Successfully built imutils


In [2]:
import os, sys
import numpy as np
import cv2
import time
from imutils.object_detection import non_max_suppression
import pickle
from PIL import Image

def east_detect(image):
    layerNames = ["feature_fusion/Conv_7/Sigmoid","feature_fusion/concat_3"]
    orig = image.copy()
    
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    
    (H, W) = image.shape[:2]
    
    # set the new width and height and then determine the ratio in change
    # for both the width and height: Should be multiple of 32
    (newW, newH) = (320, 320)
    
    rW = W / float(newW)
    rH = H / float(newH)
    
    # resize the image and grab the new image dimensions
    image = cv2.resize(image, (newW, newH))
    
    (H, W) = image.shape[:2]
    
    net = cv2.dnn.readNet("frozen_east_text_detection.pb")
    
    blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
    	(123.68, 116.78, 103.94), swapRB=True, crop=False)
    
    start = time.time()
    
    net.setInput(blob)
    
    (scores, geometry) = net.forward(layerNames)
    
    (numRows, numCols) = scores.shape[2:4]
    rects = []
    confidences = []
    # loop over the number of rows
    for y in range(0, numRows):
        # extract the scores (probabilities), followed by the geometrical
        # data used to derive potential bounding box coordinates that
        # surround text
        scoresData = scores[0, 0, y]
        xData0 = geometry[0, 0, y]
        xData1 = geometry[0, 1, y]
        xData2 = geometry[0, 2, y]
        xData3 = geometry[0, 3, y]
        anglesData = geometry[0, 4, y]
    
        for x in range(0, numCols):
    		# if our score does not have sufficient probability, ignore it
            # Set minimum confidence as required
            if scoresData[x] < 0.5:
                continue
    		# compute the offset factor as our resulting feature maps will
            #  x smaller than the input image
            (offsetX, offsetY) = (x * 4.0, y * 4.0)
            # extract the rotation angle for the prediction and then
            # compute the sin and cosine
            angle = anglesData[x]
            cos = np.cos(angle)
            sin = np.sin(angle)
            # use the geometry volume to derive the width and height of
            # the bounding box
            h = xData0[x] + xData2[x]
            w = xData1[x] + xData3[x]
            # compute both the starting and ending (x, y)-coordinates for
            # the text prediction bounding box
            endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
            endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
            startX = int(endX - w)
            startY = int(endY - h)
            # add the bounding box coordinates and probability score to
            # our respective lists
            rects.append((startX, startY, endX, endY))
            confidences.append(scoresData[x])
                        
    boxes = non_max_suppression(np.array(rects), probs=confidences)
    # loop over the bounding boxes
    for (startX, startY, endX, endY) in boxes:
# scale the bounding box coordinates based on the respective
# ratios
    	startX = int(startX * rW)
    	startY = int(startY * rH)
    	endX = int(endX * rW)
    	endY = int(endY * rH)
    	# draw the bounding box on the image
    	cv2.rectangle(orig, (startX, startY), (endX, endY), (0, 255, 0), 2)
    
    
    print(time.time() - start)
    return orig
image = cv2.imread(r"NIC Images/1.jpg")
out_image = east_detect(image)
cv2.imwrite(r"NIC_Images/1.jpg", out_image)

1.3299846649169922


True

In [4]:
out_image

array([[[167, 166, 176],
        [167, 166, 176],
        [166, 165, 175],
        ...,
        [149, 151, 159],
        [151, 150, 159],
        [151, 150, 159]],

       [[167, 166, 176],
        [167, 166, 176],
        [166, 165, 175],
        ...,
        [149, 151, 159],
        [151, 150, 159],
        [151, 150, 159]],

       [[167, 166, 176],
        [167, 166, 176],
        [166, 165, 175],
        ...,
        [149, 150, 160],
        [151, 150, 160],
        [151, 150, 160]],

       ...,

       [[148, 144, 150],
        [146, 142, 148],
        [142, 138, 144],
        ...,
        [156, 154, 160],
        [154, 154, 160],
        [154, 154, 160]],

       [[144, 147, 151],
        [144, 147, 151],
        [143, 146, 150],
        ...,
        [154, 152, 158],
        [154, 152, 158],
        [154, 152, 158]],

       [[141, 149, 149],
        [141, 149, 149],
        [142, 147, 148],
        ...,
        [154, 151, 160],
        [154, 152, 158],
        [154, 152, 158]]

In [5]:
im2 = Image.fromarray(out_image)
im2.show()